# Control The Robot with Joystick

In [ ]:
from FR3Py.interfaces import FR3Real
robot = FR3Real()

Test the interface by reading the robot joints:

In [ ]:
robot.get_state()

In [ ]:
from pyJoy.joy_daq import JoyManager
import time    
joy1  = JoyManager()
joy1.start_daq(joy_idx=0)
joy1.offset_calibration()
time.sleep(3)

In [ ]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
controller = WaypointController(kp=4)
# Read the initila pose of the robot
time.sleep(1)
state = robot.get_state()

q, dq = state['q'], state['dq']
p0 = controller.robot.getInfo(q,dq)['P_EE']
R0 = controller.robot.getInfo(q,dq)['R_EE']
T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
start_time = time.time()

x0, y0, z0 = 0.0, 0.0, 0.0 
R0 = np.eye(3)
while True:
    analog, digital = joy1.read_values()
    cmd = np.array([analog[0],analog[1],analog[2]])
    for i in range(3):
        if np.abs(cmd[i])<0.1:
            cmd[i] = 0 

    if digital[0]==0:
        x0 = x0 + cmd[0]*0.2/100
        y0 = y0 + cmd[1]*0.2/100
        z0 = z0 + cmd[2]*0.2/100
    else:
        omega_hat = np.array([[ 0,       -cmd[2],  cmd[1]],
                              [ cmd[2],   0,      -cmd[0]],
                              [-cmd[1],   cmd[0],      0]])
        R0 = R0@(np.eye(3)+omega_hat/100)

    time.sleep(0.01)
    T= T0@np.vstack([np.hstack([R0, np.array([x0,y0,z0]).reshape(3,1)]), np.array([0,0,0,1])])
    state = robot.get_state()
    q, dq = state['q'], state['dq']
    cmd = controller.compute(q,dq, T_cmd=T)
    robot.send_joint_command(cmd[0:7])

In [ ]:
analog, digital = joy1.read_values()
cmd_vx,cmd_vy,cmd_vz = analog[0],analog[1],analog[2]